In [1]:
#!/usr/bin/env python3

"""
Trying to outline out a basic transaction 
in a notebook
"""

'\nTrying to outline out a basic transaction \nin a notebook\n'

In [2]:
"""
basic imports of standard libraries,
and finding and importing provided libraries,
"""
import os, sys

if '__file__' not in globals():
    sys.path.append(os.path.dirname(os.path.abspath('').split('transactions')[0]))
    print(os.path.dirname(os.path.abspath('').split('transactions')[0]))
else:
    sys.path.append(os.path.dirname(__file__).split('transactions')[0])

print(sys.path)
import lib

C:\Users\zhvhe\Documents\GitHub\blockchain\python
['C:\\Users\\zhvhe\\Documents\\GitHub\\blockchain\\python\\transactions', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.2288.0_x64__qbz5n2kfra8p0\\python310.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.2288.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.2288.0_x64__qbz5n2kfra8p0\\lib', 'C:\\Users\\zhvhe\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0', '', 'C:\\Users\\zhvhe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages', 'C:\\Users\\zhvhe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\win32', 'C:\\Users\\zhvhe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local

In [3]:
## Setup our RPC socket.
rpc = lib.RpcSocket({ 'wallet': 'test2' })
assert rpc.check()

In [4]:
## Get a utxo for Alice.
alice_utxo = rpc.get_utxo(0)

In [5]:
## Get a change address for Alice.
alice_change_txout = rpc.get_recv(fmt='base58')
alice_pubkey_hash  = lib.decode_address(alice_change_txout['address'])

In [6]:
## Get a payment address for Bob.
bob_payment_txout = rpc.get_recv(fmt='base58')
bob_pubkey_hash   = lib.decode_address(bob_payment_txout['address'])

In [7]:
## Calculate our output amounts (fees!!!).
fee = 1000
bob_recv_value = alice_utxo['value'] // 2
alice_change_value = alice_utxo['value'] // 2 - fee

In [8]:
## The spending transaction.
atob_tx = {
    'version': 1,
    'vin': [{
        # We are unlocking the utxo from Alice.
        'txid': alice_utxo['txid'],
        'vout': alice_utxo['vout'],
        'script_sig': [],
        'sequence': 0xFFFFFFFF
    }],
    'vout': [
        {
            'value': bob_recv_value,
            'script_pubkey': ['OP_DUP', 'OP_HASH160', bob_pubkey_hash, 'OP_EQUALVERIFY', 'OP_CHECKSIG']
        },
        {
            'value': alice_change_value,
            'script_pubkey': ['OP_DUP', 'OP_HASH160', alice_pubkey_hash, 'OP_EQUALVERIFY', 'OP_CHECKSIG']
        }
    ],
    'locktime': 0
}

In [9]:
## Serialize the transaction and calculate the TXID.
atob_hex  = lib.encode_tx(atob_tx)
atob_txid = lib.hash256(bytes.fromhex(atob_hex))[::-1].hex()

In [10]:
## The redeem script is the LOTR Mines or Moria password. (TODO)
redeem_script = f"76a914{alice_utxo['pubkey_hash']}88ac"

In [11]:
## We are signing Alice's UTXO using BIP143 standard.
alice_signature = lib.sign_tx(
    atob_tx,                # The transaction.
    0,                      # The input being signed.
    alice_utxo['value'],    # The value of the utxo being spent.
    redeem_script,          # The redeem script to unlock the utxo. 
    alice_utxo['priv_key']  # The private key to the utxo pubkey hash.
)

In [12]:
## Include the arguments needed to unlock the redeem script.('melon')
atob_tx['vin'][0]['witness'] = [ alice_signature, alice_utxo['pub_key'] ]

In [13]:
print(f'''
## Pay-to-Pubkey-Hash Example ##

-- Transaction Id --
{atob_txid}

-- Alice UTXO --
     Txid : {alice_utxo['txid']}
     Vout : {alice_utxo['vout']}
    Value : {alice_utxo['value']}
     Hash : {alice_utxo['pubkey_hash']}

-- Sending to Bob --
  Address : {bob_payment_txout['address']}
    Coins : {bob_recv_value}

-- Change --
  Address : {alice_change_txout['address']}
      Fee : {fee}
    Coins : {alice_change_value}

-- Hex --
{lib.encode_tx(atob_tx)}
''')


## Pay-to-Pubkey-Hash Example ##

-- Transaction Id --
85f2e2cec9552b7279920b4103d32e9fd509babdeda668af2a9d084a2e55584d

-- Alice UTXO --
     Txid : c1006910b7023108e46b9808283db90aca0f37c0e29d9ad4719585a35232a501
     Vout : 0
    Value : 5000000000
     Hash : 90c45ba6539c0ba7fad922e96f0573ccddbcb5fd

-- Sending to Bob --
  Address : n1d2L55SdYMqmND8nvSLVYX58jg13dFoik
    Coins : 2500000000

-- Change --
  Address : mqWphuLhVcj331KfBC9Qiiqdo2tR8tJqTy
      Fee : 1000
    Coins : 2499999000

-- Hex --
0100000000010101a53252a3859571d49a9de2c0370fca0ab93d2808986be4083102b7106900c10000000000ffffffff0200f90295000000001976a914dc8861277c0f105d83007ecdb996f5f0171d799d88ac18f50295000000001976a9146daaec4b18029f3a8878de5c09c4e3ff544925fb88ac0247304402201573f8c1282bb8ee104c635e92b5d7d72383a6e5e4a7cdd9c6ebcf7cbad9231802201ad12a3ee88c481a1fc06ddf033fe766be26c38022d61d50f7588eff246530f2012103d30c3f0adf62d93c30fe841491e434759018030e30b97e2f9aef4daeebf8806300000000



In [14]:
## send it off :D

In [15]:
## redeem it :D 